# Mesh to Mesh Registration Example

ITK natively supports image-to-image registration, which is a common operation for medical images with symmetry. Another common method of storing 3D volumetric data is to represent volume surfaces as meshes. One way to get surface features from a sample mesh is to register a common atlas of correspondences and find related points on the surface of the sample mesh. In this example we seek to register two meshes using various ITK metrics and optimization techniques.

Registration classes are defined in the Python `hasi` submodule and built on top of the ITK Python wrapping. The `MeanSquaresRegistrar` and `DiffeoRegistrar` classes apply registration techniques to images derived from mesh inputs, while the `PointSetEntropyRegistrar` aims to register meshes via point set entropy metrics. Mesh registration is carried out with each class in this notebook on sample bone femur mesh data downloaded to the `examples/Data` folder.

This notebook requires the following modules, which can be either acquired via `pip` or built alongside the ITK `master` branch:
- [ITK](https://github.com/InsightSoftwareConsortium/ITK/)
- [ITKBoneEnhancement](https://github.com/InsightSoftwareConsortium/ITKBoneEnhancement)
- [ITKMeshToPolyData](https://github.com/InsightSoftwareConsortium/ITKMeshToPolyData)
- [ITKWidgets](https://github.com/InsightSoftwareConsortium/itkwidgets)

In [ ]:
import sys
!{sys.executable} -m pip install itk itkwidgets

In [2]:
# Update sys.path to reference src/ modules
import os
import copy
import importlib
from urllib.request import urlretrieve

import itk
from itkwidgets import view, checkerboard, compare
from ipywidgets import FloatProgress, Label, HBox, VBox, FloatText, ColorPicker, Button
PATTERN_COUNT = 5

module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path)
    
# Ignore wrapping warnings from itkwidgets
import warnings
warnings.filterwarnings('ignore')

In [3]:
os.makedirs('Input', exist_ok=True)
os.makedirs('Output', exist_ok=True)

In [4]:
MESH_TO_USE = '901-R'
TARGET_MESH_FILE = f'Input/{MESH_TO_USE}-mesh.vtk'
TEMPLATE_MESH_FILE = f'Input/906-R-atlas.obj'

MEANSQUARES_OUTPUT_FILE = f'Output/{MESH_TO_USE}-meansquares-registered.obj'
DIFFEO_OUTPUT_FILE = f'Output/{MESH_TO_USE}-diffeo-registered.obj'
POINTSET_OUTPUT_FILE = f'Output/{MESH_TO_USE}-pointset-registered.obj'
POINTSET_RESAMPLED_OUTPUT_FILE = f'Output/{MESH_TO_USE}-pointset-resampled.obj'

In [5]:
# Download meshes
if not os.path.exists(TARGET_MESH_FILE):
    url = 'https://data.kitware.com/api/v1/file/5f9daaba50a41e3d1924dae9/download'
    urlretrieve(url, TARGET_MESH_FILE)
if not os.path.exists(TEMPLATE_MESH_FILE):
    url = 'https://data.kitware.com/api/v1/file/608b006d2fa25629b970f139/download'
    urlretrieve(url, TEMPLATE_MESH_FILE)

In [6]:
template_mesh = itk.meshread(TEMPLATE_MESH_FILE, itk.F)
target_mesh = itk.meshread(TARGET_MESH_FILE, itk.F)

## Compare geometries with ITKWidgets

We can use `view`, `compare`, and `checkerboard` to inspect mesh and image data. Comparing the two meshes, we see that they are generally similar but do not precisely align by default. Attempting to set sample correspondences from the template mesh with this default alignment would yield a poor description of the sample surface. Registration will align the surfaces so that the two bones better coincide.

In [7]:
view(geometries=[template_mesh,target_mesh])

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'numberOfComponents': 3, 'd…

## Mesh-To-Image Conversion

The `hasi` package provides Python functions for various stages of the HASI shape analysis pipeline in its `align` module. These include static methods for creating surface meshes from sample images, deforming an atlas to a sample, iteratively refining an atlas from a population, and more.

The `align.mesh_to_image` function takes in a set of 3D meshes and outputs a set of ITK 3D images describing the meshes in a common space. The spacing, origin, and size of the images may be derived from the minimum common bounding box of the mesh or set from a reference image. The final space of the meshes is generated with a small empty buffer around the sample representation (5% in each direction).

In [8]:
from src.hasi.hasi.align import mesh_to_image

In [9]:
target_image, template_image = mesh_to_image([target_mesh, template_mesh])

In [10]:
compare(template_image,target_image)

AppLayout(children=(HBox(children=(Label(value='Link:'), Checkbox(value=False, description='cmap'), Checkbox(v…

In [11]:
checkerboard(template_image, target_image, pattern=PATTERN_COUNT)

## Run Mean Squares Image Registration

The `MeanSquaresRegistrar` class converts meshes to images and runs [Broyden-Fletcher-Goldfarb-Shanno Optimization](https://itk.org/Doxygen/html/classitk_1_1LBFGSBOptimizerv4.html) on a [BSplineTransform](https://itk.org/Doxygen/html/classitk_1_1BSplineTransform.html) to iteratively reduce the mean square error. The resulting transform is then applied to resample the target mesh into the template mesh domain.

Progress is shown with an itkwidgets display via hooks into the ITK event-observer system. The resultant mesh is returned as an object in the Python environment and may be optionally written out to a file. Iteration updates may also be printed to the output window with the optional `verbose` flag.

In [12]:
from src.hasi.hasi.meansquaresregistrar import MeanSquaresRegistrar

In [13]:
# Must instantiate a registration object to initialize optimizers
registrar = MeanSquaresRegistrar(verbose=True)

In [14]:
progress = FloatProgress(
        min=0.0,
        max=21.0,
        step=1
    )
box = HBox([
    Label('Register images'),
    progress
])
box

In [15]:
def update_progress():
    progress.value = registrar.optimizer.GetCurrentIteration()
registrar.optimizer.AddObserver(itk.IterationEvent(), update_progress)

1

In [16]:
(transform_result, mesh_result) = registrar.register(template_mesh,
                                                     target_mesh,
                                                     num_iterations=200,
                                                     convergence_factor=5e11,
                                                     gradient_convergence_tolerance=1e-35)
itk.meshwrite(mesh_result, MEANSQUARES_OUTPUT_FILE)

Iteration: 0 Metric: 0.4953344442828443 Infinity Norm: 0.0
Iteration: 0 Metric: 0.3679928041625301 Infinity Norm: 0.0
Iteration: 0 Metric: 0.3679928041625301 Infinity Norm: 0.0
Iteration: 1 Metric: 0.1382597111142386 Infinity Norm: 0.002047170989457994
Iteration: 1 Metric: 0.1382597111142386 Infinity Norm: 0.002047170989457994
Iteration: 2 Metric: 0.09620259423214085 Infinity Norm: 0.001261876484024073
Iteration: 2 Metric: 0.09620259423214085 Infinity Norm: 0.001261876484024073
Iteration: 3 Metric: 0.0810644365222312 Infinity Norm: 0.0011402423999786182
Iteration: 3 Metric: 0.0810644365222312 Infinity Norm: 0.0011402423999786182
Iteration: 4 Metric: 0.05506265821062781 Infinity Norm: 0.0008026746053761812
Iteration: 4 Metric: 0.05506265821062781 Infinity Norm: 0.0008026746053761812
Iteration: 5 Metric: 0.036229100368238225 Infinity Norm: 0.0005999657307756083
Iteration: 5 Metric: 0.036229100368238225 Infinity Norm: 0.0005999657307756083
Iteration: 6 Metric: 0.02201122948495042 Infinity

Comparison of the resulting mesh with the target shows successful registration. The template image is deformed with B-splines to approximate the shape of the target image.

In [17]:
view(geometries=[mesh_result,target_mesh])

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'numberOfComponents': 3, 'd…

## Diffeomorphic Registration

The `DiffeoRegistrar` class converts meshes to images and performs registration using the [Diffeomorphic Demons registration algorithm](https://itk.org/Doxygen/html/classitk_1_1DiffeomorphicDemonsRegistrationFilter.html). The resultant deformation field is then applied to resample the target mesh into the template mesh domain.

Custom observers may print out iteration data accessed via the `registrar.filter` object.

In [18]:
from src.hasi.hasi.diffeoregistrar import DiffeoRegistrar

In [19]:
registrar = DiffeoRegistrar(verbose=True)

In [20]:
diffeoProgress = FloatProgress(
        min=0.0,
        max=200.0,
        step=1
    )
diffeoBox = HBox([
    Label('Register images'),
    diffeoProgress
])
diffeoBox

In [21]:
def update_diff_progress():
    diffeoProgress.value = registrar.filter.GetElapsedIterations()

registrar.filter.AddObserver(itk.IterationEvent(),update_diff_progress)

1

In [22]:
(transform_result, mesh_result) = registrar.register(template_mesh,
                                        target_mesh,
                                        max_rms_error=1e-3,
                                        verbose=True)
itk.meshwrite(mesh_result, DIFFEO_OUTPUT_FILE)

Iteration: 0 Metric: 1.7976931348623157e+308 RMS Change: 1.7976931348623157e+308
Iteration: 1 Metric: 0.49533444450275316 RMS Change: 0.013359611954732018
Iteration: 2 Metric: 0.49199781461519565 RMS Change: 0.013039523388312891
Iteration: 3 Metric: 0.490142503758756 RMS Change: 0.012588951149819737
Iteration: 4 Metric: 0.48831526156735333 RMS Change: 0.012125892645666197
Iteration: 5 Metric: 0.4876264321046579 RMS Change: 0.0117265151362644
Iteration: 6 Metric: 0.48637588753497457 RMS Change: 0.011475353166185287
Iteration: 7 Metric: 0.4848329850901018 RMS Change: 0.011286081549756901
Iteration: 8 Metric: 0.48323803744259636 RMS Change: 0.01110896155680142
Iteration: 9 Metric: 0.4816613820019586 RMS Change: 0.010905596244263766
Iteration: 10 Metric: 0.4799231170176491 RMS Change: 0.010695953163007688
Iteration: 11 Metric: 0.47819615858812137 RMS Change: 0.010497861548657563
Iteration: 12 Metric: 0.47641801587190835 RMS Change: 0.01032998778603012
Iteration: 13 Metric: 0.47463753255757

Iteration: 115 Metric: 0.3063552301301444 RMS Change: 0.0072544715577085635
Iteration: 116 Metric: 0.30499025708048805 RMS Change: 0.007245972988814546
Iteration: 117 Metric: 0.3034550585458994 RMS Change: 0.007252260081452726
Iteration: 118 Metric: 0.3020681634478834 RMS Change: 0.007245775136241464
Iteration: 119 Metric: 0.300560266296785 RMS Change: 0.007251024131039496
Iteration: 120 Metric: 0.2991600045892858 RMS Change: 0.007245469877084949
Iteration: 121 Metric: 0.2976003670321141 RMS Change: 0.0072504201340405396
Iteration: 122 Metric: 0.29616745186846516 RMS Change: 0.007244353701644031
Iteration: 123 Metric: 0.2946398147859524 RMS Change: 0.007251717901203141
Iteration: 124 Metric: 0.2933072535015559 RMS Change: 0.00724372394648649
Iteration: 125 Metric: 0.29182044336183477 RMS Change: 0.0072486780913258815
Iteration: 126 Metric: 0.29041490124590047 RMS Change: 0.007244446345622724
Iteration: 127 Metric: 0.28897342135847376 RMS Change: 0.0072489073042364596
Iteration: 128 Met

Comparison of the resulting mesh with the target shows successful registration. The deformation field applied to the template mesh closely matches the shape of the target.

In [23]:
view(geometries=[mesh_result, target_mesh])

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'numberOfComponents': 3, 'd…

# Entropy-based Registration

The `PointSetEntropyRegistrar` class registers two 3D meshes by computing the transform which minimizes entropy measures between the two point clouds without need for mesh-to-image conversion.

In this example we substitute a [`EuclideanDistancePointSetToPointSetMetric`](https://itk.org/Doxygen/html/classitk_1_1EuclideanDistancePointSetToPointSetMetricv4.html) to compare the two clouds. By default an [`itk.AffineTransform`](https://itk.org/Doxygen/html/classitk_1_1AffineTransform.html) is employed for registration, but the user may subsitute a different transform inheriting from `itk.Transform` as an argument to registration.

In [24]:
from src.hasi.hasi.pointsetentropyregistrar import PointSetEntropyRegistrar
registrar = PointSetEntropyRegistrar(verbose=True)

In [25]:
entropyProgress = FloatProgress(
        min=0.0,
        max=185.0,
        step=1
    )
progressBox = HBox([
    Label('Register images'),
    entropyProgress
])
progressBox

In [26]:
def update_progress():
    entropyProgress.value = registrar.optimizer.GetCurrentIteration()

registrar.optimizer.AddObserver(itk.IterationEvent(),update_progress)

1

In [27]:
metric = itk.EuclideanDistancePointSetToPointSetMetricv4[itk.PointSet[itk.F,3]].New()

(transform_result, mesh_result) = registrar.register(
                       template_mesh=template_mesh,
                       target_mesh=target_mesh,
                       metric=metric,
                       learning_rate=1.0,
                       minimum_convergence_value=1e-6,
                       convergence_window_size=3,
                       max_iterations=300)
itk.meshwrite(mesh_result, POINTSET_OUTPUT_FILE)

Iteration: 0 Metric: 0.0
Iteration: 0 Metric: 0.0
Iteration: 0 Metric: 0.0
Iteration: 0 Metric: 0.0
Iteration: 0 Metric: 0.0
Iteration: 0 Metric: 0.0
Iteration: 0 Metric: 0.18402767981233376
Iteration: 1 Metric: 0.13636549597252293
Iteration: 2 Metric: 0.1136153451996774
Iteration: 3 Metric: 0.10070102504901322
Iteration: 4 Metric: 0.0921356833845075
Iteration: 5 Metric: 0.08639007757362922
Iteration: 6 Metric: 0.08227116489324207
Iteration: 7 Metric: 0.07907374804042967
Iteration: 8 Metric: 0.07637873779025549
Iteration: 9 Metric: 0.07405272544249199
Iteration: 10 Metric: 0.0719671439265937
Iteration: 11 Metric: 0.07005118273844133
Iteration: 12 Metric: 0.06828105853186942
Iteration: 13 Metric: 0.06660776118233713
Iteration: 14 Metric: 0.06498893467311129
Iteration: 15 Metric: 0.06343569110339238
Iteration: 16 Metric: 0.06194662338114907
Iteration: 17 Metric: 0.06055190889404482
Iteration: 18 Metric: 0.059165763253996674
Iteration: 19 Metric: 0.05782261337640995
Iteration: 20 Metric: 

Comparison of the registered template with the target mesh shows successful registration. Note that the `itk.AffineTransform` does not change the shape of the template atlas.

In [28]:
view(geometries=[mesh_result,target_mesh])

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'numberOfComponents': 3, 'd…

## Resample From Target

A common procedure for comparing correspondences across samples is to register an atlas to each mesh sample and then deform the template to align with points on the target surface. The `hasi` package provides an interface to use ITK's [`KdTree`](https://itk.org/Doxygen/html/classitk_1_1Statistics_1_1KdTree.html) to deform each template point to its nearest neighbor on the target mesh.

In [29]:
from src.hasi.hasi.align import resample_template_from_target

template_deformed = resample_template_from_target(mesh_result, target_mesh)

itk.meshwrite(template_deformed,POINTSET_RESAMPLED_OUTPUT_FILE)

In [30]:
# Use the wireframe option to examine correspondences on the deformed template
view(geometries=[template_deformed,target_mesh])

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'numberOfComponents': 3, 'd…

In [31]:
# Clean up file output
os.remove(MEANSQUARES_OUTPUT_FILE)
os.remove(DIFFEO_OUTPUT_FILE)
os.remove(POINTSET_OUTPUT_FILE)
os.remove(POINTSET_RESAMPLED_OUTPUT_FILE)